In [1]:
from Functions.Preprocess import Preprocessing
from Functions.Cluster import text_Clustering
import pandas as pd

In [2]:
# Data
incidents = pd.read_excel('C:/Users/lgorle/Desktop/UC2Ticketcategorization/teamcode/Dataset/QBE_Volumetrics_PF_V2.0.xlsx',sheet_name="Inc_PF")

In [3]:
# Ticket Title
ticket_title = incidents['Description']

###  Preprocessing Text

In [4]:
pre = Preprocessing()

In [5]:
preprocessed_data = pre.text(list(ticket_title))

time elapsed: 2.0378594398498535 sec


In [9]:
df=pd.DataFrame()
df['Desc']=preprocessed_data
df.to_excel("prepr.xlsx",index=False)

### Text to Vectors

In [7]:
# tfidf matrix and feature_names
tfid_data,terms = pre.text_vect(data = preprocessed_data)
print('shape of tfidf matrix:',tfid_data.shape,'\nlength of terms:',len(terms))

time elapsed: 0.27569150924682617 sec
shape of tfidf matrix: (839, 3192) 
length of terms: 3192


### Clustering


In [8]:
cluster = text_Clustering()

In [9]:
# Latent Semantic Analysis
Lsa = cluster.lsa(data = tfid_data,n_components=160)
variance,comp,lsa_data = Lsa.cluster_()
print('Explained Variance:',variance,'\nComponents shape:',comp.shape,'\nlsa_data shape:',lsa_data.shape)

time elapsed: 1.7662761211395264 sec
Explained Variance: 0.645459566741522 
Components shape: (160, 3192) 
lsa_data shape: (839, 160)


In [10]:
# Kmeans
clusters = cluster.Kmeans().cluster(lsa_data,160)

time elapsed: 5.351538419723511 sec


In [12]:
# Creating a final Dataframe 
final = pd.DataFrame(list(ticket_title),columns=['ticket_title'])
final['new_ticket_title'] = preprocessed_data
#final['std_event'] = incidents['Standard Event']
final['cluster'] = clusters
final.head()

,ticket_title,new_ticket_title,cluster
0,TNTX_Common Sales [PROD] - Execute scripts on ...,tntx common sales prod execute scripts prod en...,146
1,TNTX_2018 MR04: EUR1 MAIN - CWC SW Deployment ...,tntx mr eur main cwc sw deployment snow chg,42
2,TNTX_Run INSERT scripts on PROD server ukwysms...,tntx run insert scripts prod server snow chg,153
3,TNTX_2018: MR04: EUR1 Main: Global OPS: ACAS G...,tntx mr eur main global ops acas global deploy...,42
4,TNTX_2018: MR04: EUR2 MAIN: Global OPS: ACAS G...,tntx mr eur main global ops acas global deploy...,42


In [13]:
# Most_Frequent terms in a Cluster 
from collections import Counter
num_clusters = max(final.cluster)
most_freq = []
# for each cluster 
for clus in range(0,num_clusters):
    # Group by new_ticket_title 
    x = pd.DataFrame(final[final.cluster == clus].new_ticket_title)
    # appending 5 most frequent terms
    most_freq.append((Counter(' '.join(x['new_ticket_title']).split()).most_common(5),clus))

In [14]:
# Merging most frequent words with data frame
analysis = final.merge(right = pd.DataFrame(most_freq,columns = ['most_frequent_terms','cluster']),how='left',on = 'cluster' )
analysis.head()

,ticket_title,new_ticket_title,cluster,most_frequent_terms
0,TNTX_Common Sales [PROD] - Execute scripts on ...,tntx common sales prod execute scripts prod en...,146,"[(prod, 5), (chg, 5), (tntx, 4), (snow, 3), (c..."
1,TNTX_2018 MR04: EUR1 MAIN - CWC SW Deployment ...,tntx mr eur main cwc sw deployment snow chg,42,"[(tntx, 7), (mr, 7), (main, 7), (snow, 7), (ch..."
2,TNTX_Run INSERT scripts on PROD server ukwysms...,tntx run insert scripts prod server snow chg,153,"[(server, 11), (snow, 10), (chg, 10), (tntx, 9..."
3,TNTX_2018: MR04: EUR1 Main: Global OPS: ACAS G...,tntx mr eur main global ops acas global deploy...,42,"[(tntx, 7), (mr, 7), (main, 7), (snow, 7), (ch..."
4,TNTX_2018: MR04: EUR2 MAIN: Global OPS: ACAS G...,tntx mr eur main global ops acas global deploy...,42,"[(tntx, 7), (mr, 7), (main, 7), (snow, 7), (ch..."


In [39]:
# Writing data to excel
analysis.to_excel('Dataset\Analysis.xlsx',index = False)

In [1]:
[x.lower() for x in ["A","B","C"]]

['a', 'b', 'c']